This runs confirmatory analyses in pretone5 data to check for the precue pupil effects for the section "Rule- and stimulus-based biases exhibited distinct physiological signatures"

# Data setup

In [1]:
#clear memory
rm(list=ls())

### this runs regressions for pretone5 choice-aligned pupil data

## LOADING data/libraries ##

#load libraries
library(lme4)
library(lmerTest)
library(car)
library(plyr)
library(dplyr)
library(ggplot2)
#library(afex)
library(emmeans)
emm_options(lmerTest.limit = Inf, lmer.df = "satterthwaite")


switch(Sys.info()[['sysname']],
       Windows = setwd(file.path(
         Sys.getenv('USERPROFILE'),'Dropbox/Goldlab/AuditoryPriors/data processing/pupil')),
       Darwin = setwd('~/Dropbox/Goldlab/AuditoryPriors/data processing/pupil')
)

#path to data files
switch(Sys.info()[['sysname']],
       Windows = DATA_OUT_PATH <- (paste0(
         Sys.getenv('USERPROFILE'),'/OneDrive/Goldlab/AuditoryPriors/cached data/')),
       Darwin = DATA_OUT_PATH <- '~/OneDrive/Goldlab/AuditoryPriors/cached data/'
)
pd_file= paste0(DATA_OUT_PATH,'pupil_data_pretone5_ds50_forR_10-Jun-2021.csv')
baseline_file = paste0(DATA_OUT_PATH,'pupil_data_pretone5_bl_forR_10-Jun-2021.csv')


#set temporal parameters [this is easier than pupilnet cause I kept it in actual ms]
#period in which incon-con is sig (corrected) in both choice regression and bias correlation
choice_period_ms.prior <- c(220,720) 

#load data
pd_df<-read.table(pd_file,sep=',', header=TRUE, stringsAsFactors=FALSE,na.strings = c('NaN'))
bl_df<-read.table(baseline_file,sep=',', header=TRUE, stringsAsFactors=FALSE,na.strings = c('NaN'))

head(pd_df)
head(bl_df)

Loading required package: Matrix


Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step


Loading required package: carData


Attaching package: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following object is masked from ‘package:car’:

    recode


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




,trialN,trial_time_choice,trial_time_stimOn,trial_time_stimOff,pupilCblz,posXCbl,posYCbl,dataID,pupilCblz2,aSNR,isH,success,choice01,congruent,congruentpt
,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>
1,1,-2980,-260,-2560,-0.0002800095,-2.5232272,-0.7322874,1,-0.4301981,0.1,1,0,0,1,0
2,1,-2960,-240,-2540,-0.0017496437,-2.5896936,-1.0340235,1,-0.4316678,0.1,1,0,0,1,0
3,1,-2940,-220,-2520,-0.0034527749,-1.6024813,2.2097213,1,-0.4333709,0.1,1,0,0,1,0
4,1,-2920,-200,-2500,-0.0046277308,-2.8182184,1.4808445,1,-0.4345459,0.1,1,0,0,1,0
5,1,-2900,-180,-2480,-0.0051197228,-3.2270232,2.7052371,1,-0.4350378,0.1,1,0,0,1,0
6,1,-2880,-160,-2460,-0.0052058297,-0.4130138,4.3876207,1,-0.4351240,0.1,1,0,0,1,0


,trialN,GroupCount,pupilBL,posXBL,posYBL,dataID,pupilBL2,aSNR,isH,success,choice01,pretoneBias,ptC,congruent,congruentpt
,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<chr>,<int>,<int>
1,1,41,0.8024736,1191.465,682.6390,1,0.429918124,0.10,1,0,0,1.0,HH,1,0
2,2,41,1.0121036,1206.727,643.3890,1,0.535460081,0.10,0,0,1,0.2,HH,0,3
3,3,41,1.3614252,1184.631,615.0854,1,0.247339588,0.05,0,1,0,-1.0,LL,1,3
4,4,41,1.5249293,1190.676,638.3281,1,0.166733954,0.50,0,1,0,-1.0,LL,1,3
5,5,41,1.2850453,1174.691,640.6988,1,0.496517861,0.05,0,1,0,0.6,HL,1,1
6,6,41,1.5145250,1192.505,621.3683,1,0.003551195,0.10,0,1,0,-0.6,LL,1,3


In [2]:
#cleanup
bl_df$GroupCount <- NULL

#set up variables/factors
my_simple2<-contr.treatment(2,base=2) - matrix(rep(1/2,2))

pd_df$congruentpt.f <- factor(pd_df$congruentpt,levels=c(3,2,1,0),
                            labels=c("con","con-incon","incon-con","incon"))
contrasts(pd_df$congruentpt.f) <- contr.sum(4)
contrasts(pd_df$congruentpt.f)
unique(pd_df[,c('congruentpt','congruentpt.f')])

pd_df$congruent.f <- factor(pd_df$congruent,levels=c(1,0,-1),
                            labels=c("congruent","incongruent","no prior"))
contrasts(pd_df$congruent.f) <- contr.sum(3)
contrasts(pd_df$congruent.f)
unique(pd_df[,c('congruent','congruent.f')])

pd_df$isH.fs <- factor(pd_df$isH,levels=c(1,0),
                      labels=c("high","low"))
contrasts(pd_df$isH.fs) <- my_simple2
contrasts(pd_df$isH.fs)
pd_df$isH.f <- pd_df$isH.fs
contrasts(pd_df$isH.f) <- contr.sum(2)
contrasts(pd_df$isH.f)

#merge in baseline data
pd_df <- left_join(pd_df,bl_df[,c('dataID','trialN','pupilBL2')],by=c('dataID','trialN'))

#get rid of missing before scaling vars (probably already done w/ new preproc)
pd_df <- pd_df[complete.cases(pd_df),]

#get a handle on descriptives to inform scaling
scalevars_stats <- summarise(pd_df,
                             m_bl=mean(pupilBL2,na.rm=T),
                             sd_bl=sd(pupilBL2,na.rm=T),
                             m_x=mean(posXCbl,na.rm=T),
                             sd_x=sd(posXCbl,na.rm=T),
                             m_y=mean(posYCbl,na.rm=T),
                             sd_y=sd(posYCbl,na.rm=T),
                             m_asnr=mean(aSNR,na.rm=T),
                             sd_asnr=sd(aSNR,na.rm=T),
)

#set up variables: centering/scaling
pd_df$zaSNR <- scale(pd_df$aSNR)
pd_df$blz <- scale(pd_df$pupilBL2,center=T,scale=F) #not scaling since already scaled
pd_df$posX <- scale(pd_df$posXCbl)
pd_df$posY <- scale(pd_df$posYCbl)
#pd_df$zptlen <- scale(pd_df$pretoneLength)

con,1,0,0
con-incon,0,1,0
incon-con,0,0,1
incon,-1,-1,-1


,congruentpt,congruentpt.f
,<int>,<fct>
1,0,incon
226,3,con
427,1,incon-con
3040,2,con-incon


congruent,1,0
incongruent,0,1
no prior,-1,-1


,congruent,congruent.f
,<int>,<fct>
1,1,congruent
626,0,incongruent
6928,-1,no prior


,1
high,0.5
low,-0.5


high,1
low,-1


# Correct trials, precue window

In [3]:
### CORRECT TRIALS, precue window ####
pd_choicedf.cpc <- subset(pd_df,success==1 & 
                            trial_time_choice >= choice_period_ms.prior[1] & 
                            trial_time_choice <= choice_period_ms.prior[2])

pd_choicedf.cpc.ave <- summarise(group_by(pd_choicedf.cpc,
                                          dataID,trialN,
                                          zaSNR,congruent.f,congruentpt.f,isH.f),
                                 pupilCblz2=mean(pupilCblz2),
                                 blz=mean(blz),#blz could be a grouping factor but this is fine
                                 posX=mean(posX),
                                 posY=mean(posY))

#set up vars for zero corr
pd_choicedf.cpc.ave[,c('congruentpt.f1','congruentpt.f2','congruentpt.f3')] <- 
  model.matrix(~1+pd_choicedf.cpc.ave$congruentpt.f,pd_choicedf.cpc.ave)[,2:4]
pd_choicedf.cpc.ave$isH.f1 <- 
  model.matrix(~1+pd_choicedf.cpc.ave$isH.f,pd_choicedf.cpc.ave)[,2]
pd_choicedf.cpc.ave[c('congruent.f1','congruent.f2')] <- 
  model.matrix(~1+pd_choicedf.cpc.ave$congruent.f,pd_choicedf.cpc.ave)[,2:3]
unique(pd_choicedf.cpc.ave[,c("congruentpt.f","congruentpt.f1","congruentpt.f2","congruentpt.f3")])
unique(pd_choicedf.cpc.ave[,c("isH.f","isH.f1")])
unique(pd_choicedf.cpc.ave[,c("congruent.f","congruent.f1","congruent.f2")])

`summarise()` has grouped output by 'dataID', 'trialN', 'zaSNR', 'congruent.f', 'congruentpt.f'. You can override using the `.groups` argument.



congruentpt.f,congruentpt.f1,congruentpt.f2,congruentpt.f3
<fct>,<dbl>,<dbl>,<dbl>
con,1,0,0
incon-con,0,0,1
incon,-1,-1,-1
con-incon,0,1,0


isH.f,isH.f1
<fct>,<dbl>
low,-1
high,1


congruent.f,congruent.f1,congruent.f2
<fct>,<dbl>,<dbl>
congruent,1,0
incongruent,0,1
no prior,-1,-1


In [4]:
#these don't converge, so iteratively reducing

choice.lm.zc.cpc <- lmer(pupilCblz2~
                           congruent.f + congruentpt.f + zaSNR + 
                           isH.f + blz + posX + posY + 
                           (1 + congruent.f1 + congruent.f2 + 
                              congruentpt.f1 + congruentpt.f2 + congruentpt.f3 + 
                              zaSNR + isH.f1 + blz + posX + posY||dataID),
                         data=pd_choicedf.cpc.ave,
                         control=lmerControl(optimizer="bobyqa",
                                             optCtrl=list(maxfun=2e5)))

VarCorr(choice.lm.zc.cpc)

boundary (singular) fit: see ?isSingular



 Groups    Name           Std.Dev. 
 dataID    (Intercept)    0.1576116
 dataID.1  congruent.f1   0.0425269
 dataID.2  congruent.f2   0.0381597
 dataID.3  congruentpt.f1 0.0164071
 dataID.4  congruentpt.f2 0.0037054
 dataID.5  congruentpt.f3 0.0000000
 dataID.6  zaSNR          0.0326507
 dataID.7  isH.f1         0.0219964
 dataID.8  blz            0.1035137
 dataID.9  posX           0.0281598
 dataID.10 posY           0.0769590
 Residual                 0.2914817

In [5]:
choice.lm.cpc.2 <- lmer(pupilCblz2~
                             congruent.f + congruentpt.f + zaSNR + 
                             isH.f + blz + posX + posY + 
                             (1 + congruent.f1 + congruent.f2 + 
                                congruentpt.f1 + congruentpt.f2 + 
                                zaSNR + isH.f1 + blz + posX + posY|dataID),
                           data=pd_choicedf.cpc.ave,
                           control=lmerControl(optimizer="bobyqa",
                                               optCtrl=list(maxfun=2e5)))

boundary (singular) fit: see ?isSingular

Warning message:
“Model failed to converge with 1 negative eigenvalue: -2.6e+01”


In [6]:
#FINAL
#final model
choice.lm.zc.cpc.2 <- lmer(pupilCblz2~
                           congruent.f + congruentpt.f + zaSNR + 
                           isH.f + blz + posX + posY + 
                           (1 + congruent.f1 + congruent.f2 + 
                              congruentpt.f1 + congruentpt.f2 + 
                              zaSNR + isH.f1 + blz + posX + posY||dataID),
                         data=pd_choicedf.cpc.ave,
                         control=lmerControl(optimizer="bobyqa",
                                             optCtrl=list(maxfun=2e5)))

In [7]:
summary(choice.lm.zc.cpc.2)
anova(choice.lm.zc.cpc.2,type="II")
contrast(emmeans(choice.lm.zc.cpc.2,~congruent.f),adjust="holm",method="revpairwise")
contrast(emmeans(choice.lm.zc.cpc.2,~congruent.f),adjust="none",method="revpairwise")

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: pupilCblz2 ~ congruent.f + congruentpt.f + zaSNR + isH.f + blz +  
    posX + posY + (1 + congruent.f1 + congruent.f2 + congruentpt.f1 +  
    congruentpt.f2 + zaSNR + isH.f1 + blz + posX + posY || dataID)
   Data: pd_choicedf.cpc.ave
Control: lmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 2e+05))

REML criterion at convergence: 1863.8

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-8.5707 -0.5877 -0.0289  0.5565  6.6427 

Random effects:
 Groups   Name           Variance  Std.Dev.
 dataID   (Intercept)    2.484e-02 0.157612
 dataID.1 congruent.f1   1.809e-03 0.042527
 dataID.2 congruent.f2   1.456e-03 0.038160
 dataID.3 congruentpt.f1 2.692e-04 0.016407
 dataID.4 congruentpt.f2 1.374e-05 0.003706
 dataID.5 zaSNR          1.066e-03 0.032651
 dataID.6 isH.f1         4.838e-04 0.021996
 dataID.7 blz            1.072e-02 0.103514
 dataID.8 posX           7.930e-04 0.0281

,Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F)
,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
congruent.f,1.40490437,0.70245219,2,20.16797,8.2678823,2.386047e-03
congruentpt.f,0.69228270,0.23076090,3,40.84241,2.7160624,5.709277e-02
zaSNR,1.12478156,1.12478156,1,18.17428,13.2387111,1.854869e-03
isH.f,0.05343982,0.05343982,1,18.38478,0.6289882,4.378428e-01
blz,2.49678986,2.49678986,1,15.69354,29.3872879,6.050719e-05
posX,1.17320324,1.17320324,1,14.75129,13.8086356,2.123878e-03
posY,0.35016345,0.35016345,1,15.21232,4.1214338,6.020782e-02


 contrast                estimate     SE    df t.ratio p.value
 incongruent - congruent   0.0871 0.0238 43.94   3.652  0.0021
 no prior - congruent      0.0700 0.0254 15.15   2.750  0.0296
 no prior - incongruent   -0.0171 0.0295  9.52  -0.580  0.5756

Results are averaged over the levels of: congruentpt.f, isH.f 
Degrees-of-freedom method: satterthwaite 
P value adjustment: holm method for 3 tests 

 contrast                estimate     SE    df t.ratio p.value
 incongruent - congruent   0.0871 0.0238 43.94   3.652  0.0007
 no prior - congruent      0.0700 0.0254 15.15   2.750  0.0148
 no prior - incongruent   -0.0171 0.0295  9.52  -0.580  0.5756

Results are averaged over the levels of: congruentpt.f, isH.f 
Degrees-of-freedom method: satterthwaite 